---
<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2020</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1>Issued Tuesday, 3/24/2020; Due Tuesday, 4/7/2020</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for up to two class meetings after the due date. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Komal"
    last = "Essarani"
    onyen = "kessara0"
    pid = "730235611"

---
**Problem #1:** Find the longest common subsequence (LCS) between the first 100 bases and the second 100 bases of the Coronavirus genome [<a href="https://csbio.unc.edu/mcmillan/Comp555S20/data/COVID19.fa" download="COVID19.fa">SARS-CoV-2</a>]. Report both the length of the LCS as well as the subsequence. Do not include the *+* sign at the start of the genome.

In [2]:
from numpy import * 

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

In [3]:
header, sequence = loadFasta("COVID19.fa")
sequence = sequence[0][1:]
first100 = sequence[0:100]
second100 = sequence[100:200]

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

score, backtrace = findLCS(first100, second100)

def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
    
lcs = LCS(backtrace,first100,backtrace.shape[0]-1,backtrace.shape[1]-1)
        
print("length of LCS:", len(lcs)) 
print("LCS:", lcs)

length of LCS: 64
LCS: GGTTTTACCTCCCAGTAAAAAAACAATTACTGTCGTTCAAACGAACTTTATCTTTGGCTGCACT


---
**Problem #2:** Find the best *local alignment* **score** between the first 100 bases and the second 100 bases of he Coronavirus genome [<a href="https://csbio.unc.edu/mcmillan/Comp555S20/data/COVID19.fa" download="COVID19.fa">SARS-CoV-2</a>] (again ignoring the initial '+'). Assume the following alignment scoring:

* +1 for a match
* -1 for a mismatch
* -2 for an indel

In [4]:
import numpy

def LocalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j] + indel, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1] + indel, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

match = {('A','A'):  1, ('A','C'): -1, ('A','G'): -1, ('A','T'): -1,
         ('C','A'): -1, ('C','C'):  1, ('C','G'): -1, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -1, ('G','G'):  1, ('G','T'): -1,
         ('T','A'): -1, ('T','C'): -1, ('T','G'): -1, ('T','T'):  1}

s, b = LocalAlign(first100,second100,match,-2)
maxij = numpy.unravel_index(numpy.argmax(s), s.shape)
print(maxij, s[maxij])

(56, 88) 7


**Problem #3:** Give an actual *local alignment* that is consistent with your answer to **Problem #2**. 

In [10]:
def LocalAlignment(b,v,w,i,j):
    if (b[i,j] == 0):
        return['', '']
    if (b[i,j] == 3):
        result = LocalAlignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = LocalAlignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = LocalAlignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result
align = LocalAlignment(b, first100, second100, maxij[0],maxij[1])
for s,t in zip(align[0], align[1]):
    print(s, t)
print(align[0])
print(align[1])

A A
A A
C C
T T
T C
_ G
T T
C C
G T
A A
T T
C C
T T
C T
T C
T T
G G
T C
A A
G G
AACTT_TCGATCTCTTGTAG
AACTCGTCTATCTTCTGCAG


---
**Problem #4:** Find the length of the longest *embedded* common subsequence in the following: 

    "GCAAAATTGTGTACTTAAGCTTAAGGTTGATACAGCCAATCCTAAGACACCTAAGTATAAGTTTGTTCGCATTCAACCAGGACAGACTTTTTCAGTGTTA"

The longest embedded common subsequence is the longest common subsequence within in a sequence for which no symbol matches itself. As an example, consider "TAGACTAAG". The longest common subsequence is the sequence itself, "TAGACTAAG", but the longest *embedded* common subsequence is "TAG". 

In [178]:
seq = "GCAAAATTGTGTACTTAAGCTTAAGGTTGATACAGCCAATCCTAAGACACCTAAGTATAAGTTTGTTCGCATTCAACCAGGACAGACTTTTTCAGTGTTA"

def findLECS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]) and (i != j):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

score, backtrace = findLECS(seq, seq)

def LECS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LECS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LECS(b,v,i,j-1)
    else:
        return LECS(b,v,i-1,j)
    
lecs = LECS(backtrace,seq,backtrace.shape[0]-1,backtrace.shape[1]-1)
        
print("length of LECS:", len(lecs))
print("LECS:", lecs)

length of LECS: 64
LECS: AAATTGTTACTTAAGTTAAGTTAAACACCTAAGATAAGTTGTTGTTCCCAACAGACTTTTTGTT


**Problem #5:** Output the longest embedded common subsequence from the string given in **Problem #4** in the following comma separated format:

    positionInFirstSubsequence, positionInSecondSubsequence, commonSymbol
    
with one symbol per line. For "TAGACTAAG" the output could be:

    0,5,T
    1,6,A
    3,8,G


In [179]:
def LocalEmbedAlignment(b,v,w,i,j):
    if (b[i,j] == 0):
        return['', '']
    if (b[i,j] == 3):
        result = LocalEmbedAlignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        print(i-1, j-1, seq[i-1],sep=",")
        return result
    if (b[i,j] == 2):
        result = LocalEmbedAlignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = LocalEmbedAlignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result
    
max_embedded_ij = numpy.unravel_index(numpy.argmax(score), score.shape)
align = LocalEmbedAlignment(backtrace, seq, seq, max_embedded_ij[0], max_embedded_ij[1])

3,2,A
4,3,A
5,4,A
7,6,T
9,7,T
10,8,G
14,9,T
15,11,T
17,12,A
19,13,C
20,14,T
21,15,T
22,16,A
23,17,A
25,18,G
27,20,T
30,21,T
31,22,A
33,23,A
34,24,G
39,26,T
42,27,T
43,29,A
44,31,A
46,33,A
47,36,C
48,38,A
49,40,C
50,41,C
51,42,T
52,43,A
53,44,A
54,45,G
56,48,A
57,51,T
58,52,A
59,53,A
60,54,G
62,55,T
63,57,T
64,60,G
65,61,T
66,62,T
68,64,G
71,65,T
72,66,T
73,67,C
76,69,C
77,73,C
78,74,A
81,75,A
82,77,C
83,78,A
84,80,G
85,81,A
86,82,C
87,88,T
88,89,T
89,90,T
90,91,T
91,95,T
94,96,G
95,97,T
97,98,T


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.